# Compute $\chi^2$ map on $\sigma_8/\Omega_m$ parameter space for cluster count with SkySim5000 dark matter halo data

## 1. Extract DM haloes from the catalog in a given mass and redshift range.

Authors : Constantin Payerne, Céline Combet

In [ ]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
from scipy import interpolate
from astropy.table import Table
%matplotlib inline
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
Omega_sky = skysim_cat.get_catalog_info()['sky_area']
Om0 = skysim_cat.get_catalog_info()['cosmology']['Om0']
Ob0 = skysim_cat.get_catalog_info()['cosmology']['Ob0']
sigma80 = skysim_cat.get_catalog_info()['cosmology']['sigma8']
h0 = skysim_cat.get_catalog_info()['cosmology']['H0']/100
ns0 = skysim_cat.get_catalog_info()['cosmology']['n_s']

In [ ]:
cosmo_CCL_SkySim5000 = ccl.Cosmology(Omega_c=Om0 - Ob0, Omega_b=Ob0, h=h0, sigma8 = sigma80, n_s=ns0)

In [ ]:
%%time
# get list of halos in a given redshift and mass range 
mmin_extract = 1.e12 # Msun (M_fof)
zmin_extract = 0.
zmax_extract = 1.0

dm_halos = skysim_cat.get_quantities(['halo_mass','hostHaloMass','redshift','ra', 'dec', 'halo_id',
                                             'baseDC2/sod_halo_mass','baseDC2/sod_halo_radius'],
                                            filters=[f'halo_mass > {mmin_extract}','is_central==True',
                                            f'redshift>{zmin_extract}', f'redshift<{zmax_extract}'])

NB: SkySim5000 M200c masses are in units of Msun/h

In [ ]:
dm_halos['M200c'] = dm_halos['baseDC2/sod_halo_mass']/h0 # conversion M200c/h -> M200c
mask_M200c = dm_halos['M200c'] >= 0
dm_halos = Table(dm_halos)[mask_M200c]

In [ ]:
N_cl = len(dm_halos['halo_mass'])
print(f'There are {N_cl} halos in this mass (Mfof) and redshift range')

In [ ]:
plt.scatter(dm_halos['ra'], dm_halos['dec'], marker='.', s=0.001)
plt.xlabel('ra [deg]')
plt.ylabel('dec [deg]')

## 2. 2D binning of the dark matter halo catalog in the $M_{\rm 200c}-z$ plan

In [ ]:
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
z_corner = np.linspace(0.2, 0.8, 4)
Z_bin = binning(z_corner)
m_corner = np.logspace(13,14.5, 6)
Mass_bin = binning(m_corner)
m_middle = [(m_corner[i]+m_corner[i+1])/2 for i in range(len(m_corner)-1)]

In [ ]:
data, mass_edges, z_edges, im  = plt.hist2d(np.log10(dm_halos['M200c']),dm_halos['redshift'], 
                                       bins=[np.log10(m_corner),z_corner], cmin=0);
where_are_NaNs = np.isnan(data)
data[where_are_NaNs] = 0
plt.colorbar()
plt.xlabel(r'$\log_{10}(M_{\rm 200c})$', fontsize = 20)
plt.ylabel(r'$z$', fontsize = 20)
plt.show()

## 3. Computation of the Log-likelihood in the - ($\Omega_m-\sigma_8$) - plan

The cluster count prediction in redshift bin $i$ and mass bin $j$ is given by :
## $$N_{ij}^{\rm th} = \Omega_{\rm SkySim}\int_{z_i}^{z_{i+1}} dz \frac{dV(z)}{dz d\Omega}\int_{\log_{10}M_j}^{\log_{10}M_{j + 1}}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M$$

We use the Poissonian Log-Likelihood defined as the sum over all the redshift and mass bins :

## $$Log(L)(\Omega_m,\sigma_8) \propto  \sum_{i,j} N_{ij}^{\rm obs}\log( N_{ij}^{\rm th}) -  N_{ij}^{\rm th}$$

### Differential comoving volume

In [ ]:
hmd_200c = ccl.halos.MassDef200c()

def dV_over_dOmega_dz(z, cosmo):
    a = 1./(1. + z)
    da = ccl.background.angular_diameter_distance(cosmo, a) 
    E = ccl.background.h_over_h0(cosmo, a)
    return ((1+z)**2)*(da**2)*ccl.physical_constants.CLIGHT_HMPC/cosmo['h']/E

### CCL mass functions

For the example, we use the Tinker08 halo mass function from the Core Cosmology Library. The Bocquet16 halo mass function is also available for the $M_{\rm 200c}$ mass definition.

In [ ]:
def bocquet16(logm, z, cosmo):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncBocquet16(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

def tinker08(logm, z, cosmo):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

In [ ]:
def integrand_bocquet16(logm,z):
        DeltaOmega = Omega_sky * np.pi**2 / 180**2
        return DeltaOmega * bocquet16(logm, z, cosmo_CCL_SkySim5000)*dV_over_dOmega_dz(z, cosmo_CCL_SkySim5000)
    
def integrand_tinker08(logm,z):
        DeltaOmega = Omega_sky * np.pi**2 / 180**2
        return DeltaOmega * tinker08(logm, z, cosmo_CCL_SkySim5000)*dV_over_dOmega_dz(z, cosmo_CCL_SkySim5000)

In [ ]:
th = np.zeros([len(Mass_bin), len(Z_bin)])

for i, m_bin in enumerate(Mass_bin):
        
        for j, z_bin in enumerate(Z_bin):
            
            logm_down, logm_up = np.log10(m_bin)[0], np.log10(m_bin)[1]
            z_down, z_up = z_bin[0], z_bin[1]

            #th = scipy.integrate.dblquad(integrand_bocquet16, z_down, z_up, lambda x:logm_down, lambda x:logm_up, epsabs=1.e-4, epsrel=1.e-4)[0]
            
            th[i, j]= scipy.integrate.dblquad(integrand_tinker08, z_down, z_up, lambda x:logm_down, lambda x:logm_up, epsabs=1.e-4, epsrel=1.e-4)[0]

In [ ]:
c = ['k', 'b', 'g', 'r']
plt.figure(figsize=(6,6))
for i, z_bin in enumerate(Z_bin):
    plt.plot([],[], label = f'{z_bin[0]:.2f} < z < {z_bin[1]:.2f}', c = c[i])
    plt.loglog(m_middle,th[:, i], '-', color = c[i])
    plt.loglog(m_middle,data[:, i], '--', color = c[i])
label = ['CCL Thinker08 prediction', 'observed']
for i,linestyle in enumerate(['-', '--']):
    plt.plot([],[], linestyle, color = 'k', label = label[i])
plt.xlabel('$M_{200c}$ [M$_\odot$]', size=14)
plt.ylabel('Number of haloes', size=14)
plt.legend(frameon = False, fontsize = 14)
plt.show()

### Poissonian Log-Likelihood

In [ ]:
def logLikelihood_dbquad(Omegam, sigma8):
    
    cosmo = ccl.Cosmology(Omega_c=Omegam - Ob0, Omega_b=Ob0, h=h0, sigma8 = sigma8, n_s=ns0)
    
    logL_poissonian = 0
    logL_Gaussian = 0
    
    cluster_abundance_obs = np.zeros([len(Mass_bin), len(Z_bin)])
    cluster_abundance_th = np.zeros([len(Mass_bin), len(Z_bin)])
    
    def integrand_bocquet16(logm,z):
        DeltaOmega = Omega_sky * np.pi**2 / 180**2
        return DeltaOmega * bocquet16(logm, z, cosmo)*dV_over_dOmega_dz(z, cosmo)
    
    def integrand_tinker08(logm,z):
        DeltaOmega = Omega_sky * np.pi**2 / 180**2
        return DeltaOmega * tinker08(logm, z, cosmo)*dV_over_dOmega_dz(z, cosmo)
    
    for i, m_bin in enumerate(Mass_bin):
        
        for j, z_bin in enumerate(Z_bin):
            
            logm_down, logm_up = np.log10(m_bin)[0], np.log10(m_bin)[1]
            z_down, z_up = z_bin[0], z_bin[1]

            #th = scipy.integrate.dblquad(integrand_bocquet16, z_down, z_up, lambda x:logm_down, lambda x:logm_up, epsabs=1.e-4, epsrel=1.e-4)[0]
            
            th= scipy.integrate.dblquad(integrand_tinker08, z_down, z_up, lambda x:logm_down, lambda x:logm_up, epsabs=1.e-4, epsrel=1.e-4)[0]
            
            obs = data[i,j]
            
            logL_poissonian = logL_poissonian + obs*np.log(th) - th
            
            logL_Gaussian = logL_Gaussian - 0.5*(obs-th)**2/obs
            
    return logL_poissonian, logL_Gaussian

In [ ]:
N_points_Omegam = 10
N_points_sigma8 = 10

Omegam_random = np.linspace(0.2, 0.3, N_points_Omegam)
sigma8_random = np.linspace(0.68, 0.9, N_points_sigma8)

lnL_map_Poissonian = np.zeros([N_points_Omegam, N_points_sigma8])
lnL_map_Gaussian = np.zeros([N_points_Omegam, N_points_sigma8])

In [ ]:
%%time

print('computing logLikelihood map ...')

for i, Om in enumerate(Omegam_random):
    
    for j, s8 in enumerate(sigma8_random):
        
        lnL_map_res_Poissonian, lnL_map_res_Gaussian  = logLikelihood_dbquad(Om, s8)
        lnL_map_Poissonian[i, j] = lnL_map_res_Poissonian
        lnL_map_Gaussian[i, j] = lnL_map_res_Gaussian

In [ ]:
logLikelihood_data_Poissonian = {'lnL_map': lnL_map_Poissonian, 
                    'Omegam' : Omegam_random, 
                    'sigma8' : sigma8_random, 
                    'cluster_abundance' : data,
                    'redshift_bin' : Z_bin,
                    'Mass_bin' : Mass_bin,}

logLikelihood_data_Gaussian = {'lnL_map': lnL_map_Gaussian, 
                    'Omegam' : Omegam_random, 
                    'sigma8' : sigma8_random, 
                    'cluster_abundance' : data,
                    'redshift_bin' : Z_bin,
                    'Mass_bin' : Mass_bin,}

## 4. Plot $\chi^2 - \chi^2_{\rm min}$ map

Make a 2d interpolation of the $\chi^2  = -2\log(L)$ map. To plot confidence contours on the ($\Omega_m -\sigma_8$) plan, we make a 2D interpolation of $\chi^2 - \chi^2_{\rm min}$.

In [ ]:
def make_chi2_interpolation(lnL_map = 1):
    
    x_down, x_up = min(lnL_map['Omegam']), max(lnL_map['Omegam'])
    y_down, y_up = min(lnL_map['sigma8']), max(lnL_map['sigma8'])
    
    r"plot chi2_map"
    XX, YY = np.meshgrid(lnL_map['Omegam'], lnL_map['sigma8'])

    chi2_map = -2*lnL_map['lnL_map'].T
    minimum = min(chi2_map.flatten())

    r"plot interpolated chi2_map"
    chi2 = interpolate.interp2d(XX, YY, chi2_map - minimum, kind = 'cubic')
    
    omegam = np.linspace(x_down, x_up, 100)
    s8 = np.linspace(y_down, y_up, 100)
    OM, S8 = np.meshgrid(omegam, s8)
    chi_interp = chi2(omegam, s8)

    m = {'chi2':chi2, 
         'chi2_map':[XX, YY, chi2_map - minimum],
         'chi2_map_interp':[OM, S8, chi_interp]}
    
    return m

In [ ]:
chi2_Poissonian = make_chi2_interpolation(lnL_map = logLikelihood_data_Poissonian)
chi2_Gaussian = make_chi2_interpolation(lnL_map = logLikelihood_data_Gaussian)

## 5. Plot $\chi^2$ map versus the cosmoDC2 cosmology

In [ ]:
plt.figure(figsize = (10,10))
plt.rcParams['axes.linewidth']= 2
plt.tick_params(axis='both', which = 'major', labelsize= 15, zorder = 0)

plt.contour(chi2_Poissonian['chi2_map_interp'][0], 
                       chi2_Poissonian['chi2_map_interp'][1], 
                       chi2_Poissonian['chi2_map_interp'][2], 
                       colors = 'r', levels = list(np.arange(1,4)**2), linestyles = 'solid')

plt.contour(chi2_Gaussian['chi2_map_interp'][0], 
                       chi2_Gaussian['chi2_map_interp'][1], 
                       chi2_Gaussian['chi2_map_interp'][2], 
                       colors = 'k', levels = list(np.arange(1,4)**2), linestyles = 'solid')

plt.ylim(min(sigma8_random), max(sigma8_random))
plt.xlim(min(Omegam_random), max(Omegam_random))

plt.vlines(Om0, 0,2, linestyle = '--', color = 'r', linewidth = 3)
plt.hlines(sigma80, 0,2, linestyle = '--', color = 'r', linewidth = 3)

plt.plot([], [], 'r', label = r'Poissonian')
plt.plot([], [], 'k', label = r'Gaussian')
plt.plot([], [], 'b', )
plt.ylabel(r'$\sigma_8$', fontsize = 30)
plt.xlabel(r'$\Omega_m$', fontsize = 30)
plt.legend(frameon = False, fontsize = 20)
plt.show()